# TOA composite creation for every Liss4 training dataset

In [ ]:
folder = r"<-- INSERT PATH HERE -->"
import os
for x in os.listdir(folder_zips):
    if not x.endswith(".zip"):
        if not x.endswith(".docx"):
            print(x)

In [ ]:
folder_list = [r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->"]

## Folder list with all the folder paths of training dataset for TOA composite creation

In [ ]:
import datetime
import ephem
import math
import os
import rioxarray as rxr
import xarray as xr
import zipfile

In [ ]:
from dask.distributed import Client, progress
client = Client() 
client

In [ ]:
def toa_conversion(folder):
    foldername = folder
    
    b2_path = os.path.join(foldername, 'BAND2.tif')
    b3_path = os.path.join(foldername, 'BAND3.tif')
    b4_path = os.path.join(foldername, 'BAND4.tif')
     
    b2_ds = rxr.open_rasterio(b2_path, chunks=True)
    b3_ds = rxr.open_rasterio(b3_path, chunks=True)
    b4_ds = rxr.open_rasterio(b4_path, chunks=True)
    
    metadata_filename = 'BAND_META.txt'
    metadata_filepath = os.path.join(foldername, metadata_filename)
     
    metadata = {}
    with open(metadata_filepath) as f:
      for line in f:
        key, value = line.split('=')
        metadata[key.strip()] = value.strip()
     
    scene_id = metadata['OTSProductID']
    print(f'Metadata extracted successfully for scene: {scene_id}')
    
    scene = xr.concat(
        [b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    )
     
    scene = scene.where(scene != 0)
    scene.name = scene_id
    # scene
    
    scene = xr.concat(
        [b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    )
     
    scene = scene.where(scene != 0)
    scene.name = scene_id
    # scene
    
    acq_date_str = metadata['DateOfPass']
    # Date is in the format 04-MAR-2023
    acq_date = datetime.datetime.strptime(
        acq_date_str, '%d-%b-%Y')
     
    sun_elevation_angle = metadata['SunElevationAtCenter']
    sun_zenith_angle = 90 - float(sun_elevation_angle)
    
    observer = ephem.Observer()
    observer.date = acq_date
    sun = ephem.Sun()
    sun.compute(observer)
    d = sun.earth_distance
    
    b2_sr = float(metadata['B2_Lmax'])
    b3_sr = float(metadata['B3_Lmax'])
    b4_sr = float(metadata['B4_Lmax'])
    
    sensor =  metadata['SatID']
    if sensor == 'IRS-R2':
        b2_esun = 185.33
        b3_esun = 157.766
        b4_esun = 111.359
    
    elif sensor == 'IRS-R2A':
        b2_esun = 185.347
        b3_esun = 158.262
        b4_esun = 110.81
    
    pi = math.pi
    sun_zenith_angle_rad = math.radians(sun_zenith_angle)
    
    b2_dn = scene.sel(band='BAND2')
    b3_dn = scene.sel(band='BAND3')
    b4_dn = scene.sel(band='BAND4')
    
    print(b2_sr)
    
    # print(b2_dn)
    b2_rad = b2_dn*b2_sr<-- INSERT PATH HERE -->
    b3_rad = b3_dn*b3_sr<-- INSERT PATH HERE -->
    b4_rad = b4_dn*b4_sr<-- INSERT PATH HERE -->
    
    b2_ref = (pi*b2_rad*d*d)/(b2_esun*math.cos(sun_zenith_angle_rad))
    b3_ref = (pi*b3_rad*d*d)/(b3_esun*math.cos(sun_zenith_angle_rad))
    b4_ref = (pi*b4_rad*d*d)/(b4_esun*math.cos(sun_zenith_angle_rad))
    
    reflectance_bands = [b2_ref, b3_ref, b4_ref]
    scene_ref = xr.concat(
        reflectance_bands, dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    ).chunk('auto')
    scene_ref.name = scene_id
    
    output_ds = scene_ref.to_dataset('band')
    
    output_file = f'test_{scene_id}.tif'
    
    output_file =os.path.join(foldername, output_file)
    output_options = {
        'driver': 'COG',
        'compress': 'deflate',
        'num_threads': 'all_cpus',
        'windowed': False 
    }
    #Save the raster.
    
    output_ds[['BAND2', 'BAND3', 'BAND4']].rio.to_raster(
        output_file, **output_options)
    print(f'Output file created {output_file}')

In [ ]:
for folder in folder_list:
    toa_conversion(folder)

## Saving the toa composite in the respective folders

# Classification for complete training data

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean
from scipy.stats import entropy
import numpy as np

import geopandas as gpd
import pandas as pd

# === Step 1: Load from the combined training GeoJSON file ===
file_path = "<-- INSERT PATH HERE -->" ## geojson file with samples of all the training dataset combined for all the three classes
#to get complete training samples
combined = gpd.read_file(file_path)

# Ensure correct dtype
combined['Id'] = combined['Id'].astype(int)

# Identify band columns (assumes only 3 bands, and filters out non-band columns)
band_cols = [col for col in combined.columns if col not in ['geometry', 'Id', 'layer', 'path']]

# Optional: Show identified band columns
print("Identified band columns:", band_cols)

In [ ]:
combined_new = combined.copy()

G = combined_new['SAMPLE_1']
R = combined_new['SAMPLE_2']
N = combined_new['SAMPLE_3']

combined_new['NDVI'] = (N - R) / (N + R + 1e-6)
combined_new['GNIRR'] = G / (N + 1e-6)
combined_new['RGR'] = R / (G + 1e-6)
combined_new['BRIGHTNESS'] = (G + R + N) / 3
combined_new['ExG'] = 2 * G - R - N
combined_new['NGRDI'] = (G - R) / (G + R + 1e-6)
combined_new['INV_NDVI'] = 1 - combined_new['NDVI']

combined_new['NDSI_shadow'] = (combined_new['BRIGHTNESS'] - N) / (combined_new['BRIGHTNESS'] + N  + 1e-6)



In [ ]:
# Identify band columns (assumes only 3 bands)
band_cols = [col for col in combined_new.columns if col not in ['geometry', 'Id','layer','path']]

# === Step 2: Plot histograms for each band by class ===
for band in band_cols:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=combined_new, x=band, hue='Id', kde=True, palette='Set1', bins=30)
    plt.title(f'Histogram of {band} by Class')
    plt.show()

# === Step 3: Compute statistical summaries ===
print("<-- INSERT PATH HERE --> Summary by Class:")
for cls in sorted(combined_new['Id'].unique()):
    print(f"<-- INSERT PATH HERE --> {cls}:")
    print(combined_new[combined_new['Id'] == cls][band_cols].describe())

# === Step 4: Class Separability ===

# -- Fisher Discriminant Ratio (FDR) for each band
def fdr(class0, class1):
    mean_diff_sq = (class0.mean() - class1.mean())**2
    
    var_sum = class0.var() + class1.var()
    return mean_diff_sq / var_sum if var_sum != 0 else 0

print("<-- INSERT PATH HERE --> Discriminant Ratios:")
classes = sorted(combined_new['Id'].unique())
for i in range(len(classes)):
    for j in range(i+1, len(classes)):
        print(f"<-- INSERT PATH HERE --> Class {classes[i]} and {classes[j]}:")
        for band in band_cols:
            fdr_value = fdr(combined_new[combined_new['Id'] == classes[i]][band],
                            combined_new[combined_new['Id'] == classes[j]][band])
            print(f"  {band}: {fdr_value:.4f}")

# -- Bhattacharyya distance (per band)
def bhattacharyya_distance(x, y):
    mu1, mu2 = x.mean(), y.mean()
    var1, var2 = x.var(), y.var()
    term1 = 0.25 * np.log(0.25 * ((var1<-- INSERT PATH HERE -->) + (var2<-- INSERT PATH HERE -->) + 2))
    term2 = 0.25 * ((mu1 - mu2) ** 2) / (var1 + var2)
    return term1 + term2

print("<-- INSERT PATH HERE --> Distances:")
for i in range(len(classes)):
    for j in range(i+1, len(classes)):
        print(f"<-- INSERT PATH HERE --> Class {classes[i]} and {classes[j]}:")
        for band in band_cols:
            bd = bhattacharyya_distance(combined_new[combined_new['Id'] == classes[i]][band],
                                        combined_new[combined_new['Id'] == classes[j]][band])
            print(f"  {band}: {bd:.4f}")

## MLP on PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === Step 1: Load from the combined training data GeoJSON file ===
file_path = "<-- INSERT PATH HERE -->"
#to get complete training samples
combined = gpd.read_file(file_path)

# Ensure correct dtype
combined['Id'] = combined['Id'].astype(int)

X = combined[["SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]].values.astype(np.float32)
y = combined["Id"].values.astype(np.int64)

# Remove rows with NaN
not_nan_idx = ~np.isnan(X).any(axis=1)
X = X[not_nan_idx]
y = y[not_nan_idx]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Dataset class ---

class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = MyDataset(X_scaled, y)

# --- Model ---

class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.net(x)

input_dim = X_scaled.shape[1]
num_classes = len(np.unique(y))
model = MLP(input_dim, num_classes).to(device)

# --- Class weights for imbalance ---

class_weights_np = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights_np, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)

# --- Train-test split ---

train_idx, test_idx = train_test_split(
    np.arange(len(dataset)),
    stratify=y,
    test_size=0.2,
    random_state=42
)

batch_size = 220

train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

# --- Optimizer ---

optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Training with Early Stopping ---

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, preds = outputs.max(1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)
    return running_loss / total, correct / total

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            running_loss += loss.item() * X_batch.size(0)
            _, preds = outputs.max(1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    return running_loss / total, correct / total

# Early stopping parameters
best_val_loss = float('inf')
patience = 20
patience_counter = 0
num_epochs = 500

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_epoch(model, test_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}, Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best weights
model.load_state_dict(best_model_wts)

# --- Evaluate on validation set ---

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, preds = outputs.max(1)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.cpu().numpy())

print("<-- INSERT PATH HERE -->=== Classification Report (Validation Set) ===")
print(classification_report(y_true, y_pred))

print("<-- INSERT PATH HERE -->=== Confusion Matrix (Validation Set) ===")
print(confusion_matrix(y_true, y_pred))

# --- Plot training loss curve ---

plt.figure(figsize=(10,6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Curve")
plt.legend()
plt.grid(True)
plt.show()


# --- Plot accuracy curve ---
plt.figure(figsize=(10,6))
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy Curve")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Curve")
plt.legend()
plt.grid(True)

plt.savefig("training_validation_loss_curve.png", dpi=300)  

plt.show()

In [ ]:
# --- Plot accuracy curve ---
plt.figure(figsize=(10,6))
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy Curve")
plt.legend()
plt.grid(True)
plt.savefig("training_validation_accuracy_curve.png", dpi=300)  
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Validation Set)")
plt.savefig("Validation_set_confusion_matrix.png", dpi=300)  
plt.show()


## Saving the mlp classifier

In [ ]:

import joblib
import os

# Save model weights
torch.save(best_model_wts, r"<-- INSERT PATH HERE -->")

# Save scaler
joblib.dump(scaler, r"<-- INSERT PATH HERE -->")

# Save model config
model_config = {
    "input_dim": input_dim,
    "num_classes": num_classes
}
joblib.dump(model_config, r"<-- INSERT PATH HERE -->")

print(f"Model and scaler saved ")


## Model architecture diagram

In [ ]:
import graphviz

# Define the MLP architecture
layers = [
    ("Input", "Input Layer<-- INSERT PATH HERE -->(3 features)"),
    ("L1", "Linear (3 → 256)"),
    ("R1", "ReLU"),
    ("D1", "Dropout (0.2)"),
    ("L2", "Linear (256 → 128)"),
    ("R2", "ReLU"),
    ("D2", "Dropout (0.2)"),
    ("L3", "Linear (128 → 64)"),
    ("R3", "ReLU"),
    ("D3", "Dropout (0.2)"),
    ("L4", "Linear (64 → 32)"),
    ("R4", "ReLU"),
    ("L5", "Linear (32 → 3)<-- INSERT PATH HERE -->(Output Layer)")
]

# Create the diagram
dot = graphviz.Digraph(format='png')
dot.attr(rankdir='LR')  # Left to right layout

# Add nodes
for key, label in layers:
    dot.node(key, label, shape='record', style='filled', fillcolor='lightblue')

# Connect the layers
for i in range(len(layers) - 1):
    dot.edge(layers[i][0], layers[i + 1][0])

# Render the diagram to a file
dot.render('mlp_model_architecture', view=True)  


In [ ]:
import graphviz

# Define the MLP architecture
layers = [
    ("Input", "Input Layer<-- INSERT PATH HERE -->(3 features)"),
    ("L1", "Linear (3 → 256)"),
    ("R1", "ReLU"),
    ("D1", "Dropout (0.2)"),
    ("L2", "Linear (256 → 128)"),
    ("R2", "ReLU"),
    ("D2", "Dropout (0.2)"),
    ("L3", "Linear (128 → 64)"),
    ("R3", "ReLU"),
    ("D3", "Dropout (0.2)"),
    ("L4", "Linear (64 → 32)"),
    ("R4", "ReLU"),
    ("L5", "Linear (32 → 3)<-- INSERT PATH HERE -->(Output Layer)")
]

# Create the diagram
dot = graphviz.Digraph(format='png')
dot.attr(rankdir='TB')  # Top to Bottom layout

# Add nodes
for key, label in layers:
    dot.node(key, label, shape='record', style='filled', fillcolor='lightblue')

# Connect the layers
for i in range(len(layers) - 1):
    dot.edge(layers[i][0], layers[i + 1][0])

# Render and view
dot.render('mlp_model_architecture_vertical', view=True)
